In [28]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
import cv2

from keras.applications import inception_v3
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input as inception_v3_preprocessor

from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import optimizers

from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

from sklearn.model_selection import train_test_split

from tqdm import tqdm

from os import makedirs
from os.path import expanduser, exists, join

In [32]:
batch_size = 16
img_width, img_height = 299, 299
num_class = 2
n_epoches = 100

print "num_classes are " , num_class

num_classes are  2


In [33]:
train_data_dir = "../data/train"
valid_data_dir = "../data/valid"

In [34]:
train_datagen = ImageDataGenerator(rescale=1./255, 
#                                    rotation_range=30, 
                                   width_shift_range=0.2,
                                   height_shift_range=0.2, 
                                   horizontal_flip = 'true')

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_height, img_width),
                                                    batch_size=batch_size,
                                                    seed=10)

val_datagen = ImageDataGenerator(rescale=1. / 255)

valid_generator = val_datagen.flow_from_directory(valid_data_dir,
                                                target_size=(img_height, img_width),
                                                batch_size=batch_size,
                                                seed=10)

print ("train data: ", train_generator.n)
print ("valid data: ", valid_generator.n)

Found 204 images belonging to 2 classes.
Found 46 images belonging to 2 classes.
('train data: ', 204)
('valid data: ', 46)


In [35]:
base_model = InceptionV3(weights = 'imagenet', include_top = False, input_shape=(img_height, img_width, 3))

In [36]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_189 (Conv2D)             (None, 149, 149, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_189 (BatchN (None, 149, 149, 32) 96          conv2d_189[0][0]                 
__________________________________________________________________________________________________
activation_189 (Activation)     (None, 149, 149, 32) 0           batch_normalization_189[0][0]    
__________________________________________________________________________________________________
conv2d_190

In [37]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

In [38]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_189 (Conv2D)             (None, 149, 149, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_189 (BatchN (None, 149, 149, 32) 96          conv2d_189[0][0]                 
__________________________________________________________________________________________________
activation_189 (Activation)     (None, 149, 149, 32) 0           batch_normalization_189[0][0]    
__________________________________________________________________________________________________
conv2d_190

In [39]:
for layer in base_model.layers:
    layer.trainable = False

In [40]:
model.compile(optimizers.adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_generator,
                      steps_per_epoch = train_generator.n // batch_size,
                      validation_data = valid_generator,
                      validation_steps = valid_generator.n // batch_size,
                      epochs = n_epoches,
                      verbose = 1)

Epoch 1/100
12/12 [==============================] - 71s 6s/step - loss: 0.7108 - acc: 0.5678 - val_loss: 0.8758 - val_acc: 0.4375
Epoch 2/100
12/12 [==============================] - 66s 6s/step - loss: 0.6652 - acc: 0.5799 - val_loss: 0.7033 - val_acc: 0.5625
Epoch 3/100
12/12 [==============================] - 62s 5s/step - loss: 0.5882 - acc: 0.7083 - val_loss: 0.7178 - val_acc: 0.5625
Epoch 4/100
12/12 [==============================] - 59s 5s/step - loss: 0.5521 - acc: 0.7279 - val_loss: 0.7446 - val_acc: 0.5625
Epoch 5/100
12/12 [==============================] - 61s 5s/step - loss: 0.5345 - acc: 0.7532 - val_loss: 0.7637 - val_acc: 0.5625
Epoch 6/100
12/12 [==============================] - 62s 5s/step - loss: 0.5060 - acc: 0.7760 - val_loss: 0.7754 - val_acc: 0.5625
Epoch 7/100
12/12 [==============================] - 60s 5s/step - loss: 0.4992 - acc: 0.7866 - val_loss: 0.7540 - val_acc: 0.5625
Epoch 8/100
12/12 [==============================] - 61s 5s/step - loss: 0.4515 - a

12/12 [==============================] - 61s 5s/step - loss: 0.2967 - acc: 0.8784 - val_loss: 1.5972 - val_acc: 0.5938
Epoch 64/100
12/12 [==============================] - 61s 5s/step - loss: 0.2681 - acc: 0.8941 - val_loss: 1.1734 - val_acc: 0.5938
Epoch 65/100
12/12 [==============================] - 61s 5s/step - loss: 0.3027 - acc: 0.8852 - val_loss: 0.5148 - val_acc: 0.7188
Epoch 66/100
12/12 [==============================] - 61s 5s/step - loss: 0.3413 - acc: 0.8383 - val_loss: 1.0643 - val_acc: 0.5938
Epoch 67/100
12/12 [==============================] - 66s 5s/step - loss: 0.2587 - acc: 0.8840 - val_loss: 1.0257 - val_acc: 0.5938
Epoch 68/100
12/12 [==============================] - 62s 5s/step - loss: 0.2588 - acc: 0.8958 - val_loss: 1.2558 - val_acc: 0.5938
Epoch 69/100
12/12 [==============================] - 60s 5s/step - loss: 0.3109 - acc: 0.8907 - val_loss: 0.9510 - val_acc: 0.5938
Epoch 70/100
12/12 [==============================] - 61s 5s/step - loss: 0.2828 - acc: 0

In [ ]:
model.save_weights("jbm_inception_weights_35.h5")